<a href="https://colab.research.google.com/github/RafsanJany-44/Research-NREM-REM/blob/main/Roc_for_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def ROC(mod,model_name,X_t,y_t,graph_feat):
  r_probs = [0 for _ in range(len(y_t))]

  model_probs = []


  for i in mod:
    model_probs.append(i.predict_proba(X_t))


  model_probs2=[]

  for i in model_probs:
    model_probs2.append(i[:,1])


  from sklearn.metrics import roc_curve, roc_auc_score

  model_auc=[]
  r_auc = roc_auc_score(y_t, r_probs)

  for i in model_probs2:
    model_auc.append(roc_auc_score(y_t,i))


  fpr_tpr=[]
  r_fpr, r_tpr, _ = roc_curve(y_t, r_probs)

  for i in model_probs2:
    fpr, tpr, _ = roc_curve(y_t, i)
    fpr_tpr.append((fpr,tpr))

  import matplotlib.pyplot as plt
  plt.plot(r_fpr, r_tpr, linestyle='--', label='Random prediction (AUROC = %0.3f)' % r_auc)

  for i in range(len(model_auc)):
    plt.plot(fpr_tpr[i][0], fpr_tpr[i][1], marker='.', label=model_name[i]+' (AUROC = %0.3f)' % model_auc[i])

  # Title
  plt.title(graph_feat["Title"],fontsize= graph_feat["Title Size"], fontweight=graph_feat["Title Fontweight"])
  plt.xlabel('False Positive Rate',fontweight=graph_feat["X axis Label Fontweight"],fontsize=graph_feat["X axis Label Font Size"])
  plt.ylabel('True Positive Rate',fontweight=graph_feat["Y axis Label Fontweight"],fontsize=graph_feat["Y axis Label Font Size"])
  
  plt.legend( 
           prop = {'size' : graph_feat["legend Font Size"]}, 
           loc = graph_feat["legend Position"])
  
  fig = plt.gcf()
  fig.set_size_inches(graph_feat["Fig Width (inches)"], graph_feat["Fig Height (inches)"])
  fig.savefig(graph_feat["Fig Saving Name"], dpi=graph_feat["dpi"])
  plt.show()

In [ ]:
graph_feat={
    "Title" : "ROC Plot Testing",
    "Title Size":18,
    "Title Fontweight":'bold',

    "legend Font Size": 14,
    "legend Position": "lower right",

    "X axis Label Font Size":16,
    "X axis Label Fontweight":'bold',

    "Y axis Label Font Size":16,
    "Y axis Label Fontweight":'bold',

    "Fig Height (inches)":10.8,
    "Fig Width (inches)":18.8,

    "Fig Saving Name": "ROC_training.png",
    "dpi":100
    

}


In [ ]:
import numpy as np
import pandas as pd

dataset=pd.read_csv("https://raw.githubusercontent.com/RafsanJany-44/Research-NREM-REM/main/dataset/REM_NREM.csv")


target="Sleep_Stage"

classes = np.array(sorted(list(set(dataset[target]))))

In [ ]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)

  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

In [ ]:
from joblib import Parallel, delayed
import joblib
  

def save_model(model,model_name):
 joblib.dump(model, model_name)
  
def load_model(model_name):
  return joblib.load(model_name)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

In [ ]:
X = dataset.loc[:, dataset.columns!=target]
y = dataset[target]

In [ ]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
new_dataset=balance(X_train,y_train)

X_train = new_dataset.loc[:, new_dataset.columns!=target]
y_train = new_dataset[target]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = load_model("/content/drive/MyDrive/MLModel_NREM_REM/RandomForestClassifier(max_depth=39, n_estimators=450, random_state=0)")

In [ ]:
y_pred_train=rf.predict(X_train)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train,y_pred_train))
print("Accurecy: ",accuracy_score(y_train, y_pred_train))

In [ ]:
y_pred_test=rf.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test,y_pred_test))
print("Accurecy: ",accuracy_score(y_test, y_pred_test))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=load_model("/content/drive/MyDrive/MLModel_NREM_REM/GradientBoostingClassifier(max_depth=12, n_estimators=1150, random_state=0)")

In [ ]:
y_pred_train=gb.predict(X_train)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train,y_pred_train))
print("Accurecy: ",accuracy_score(y_train, y_pred_train))

In [ ]:
y_pred_test=gb.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test,y_pred_test))
print("Accurecy: ",accuracy_score(y_test, y_pred_test))

In [ ]:
import xgboost as xgb
xgb =  load_model("/content/drive/MyDrive/MLModel_NREM_REM/XGBClassifier(max_depth=29, n_estimators=4010)")

In [ ]:
y_pred_test = xgb.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test,y_pred_test))
print("Accurecy: ",accuracy_score(y_test, y_pred_test))

In [ ]:
y_pred_train = xgb.predict(X_train)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train,y_pred_train))
print("Accurecy: ",accuracy_score(y_train, y_pred_train))